In [7]:
using Distributions, ForwardDiff, LinearAlgebra, Gadfly, Cairo, Fontconfig

include("../precipFramework/iGMRF.jl");
include("../precipFramework/dataGen.jl");
include("../precipFramework/utils.jl");
include("../precipFramework/plotting.jl");
include("CAVI.jl");

In [59]:
# Seed
Random.seed!(400);
# Dimensions de la grille
m₁ = 10;
m₂ = 10;
# Nb total de cellules
m = m₁ * m₂;
# Matrice de structure
κᵤ = 10.0;
F = iGMRF(m₁, m₂, κᵤ);
# Grille cible
grid_target = generateTargetGrid(F);
grid_target[:, :, 1] = grid_target[:, :, 1] .+ 10;
# Génération de données
nobs = 100;
Y = generateData(grid_target, nobs);

# Itérations

In [60]:
n_epoch = 100;
epoch_size = 1;
Hθ₀ = [fill(0.0, m)..., 1.0];

MCKL, Hθ = runCAVI(n_epoch, epoch_size, Hθ₀, F=F, Y=Y);

Minimization made in 5.260883084 s


In [ ]:
plotConvergenceCriterion(
    MCKL,
    # "../plots/mean_field/$(m₁)x$(m₂)/convergence_criterion.png",
)

In [62]:
η = Hθ[1:m];
bᵤ = Hθ[m + 1];
aᵤ = compute_aᵤ(m);
s² = compute_s²(aᵤ, bᵤ, length(Y[1]), [diag(F.G.W)...]);

In [63]:
include("../precipFramework/mcmc.jl");

niter = 10000
δ² = 0.2
κᵤ₀ = 10
μ₀ = zeros(m)

θ = gibbs(niter, Y, m₁=m₁, m₂=m₂, δ²=δ², κᵤ₀=κᵤ₀, μ₀=μ₀);

warming_size = Int(0.2 * niter);
θsampling = θ[warming_size:end, :, :];

In [ ]:
# Pour κᵤ

approxDensity(x::Real) = pdf(Gamma(aᵤ, 1/bᵤ), x)
a = 0;
step = 0.1;
b = 100;

plotApproxVSMCMC(
    approxDensity,
    θsampling.value[:, 1, 1],
    # "../plots/mean_field/$(m₁)x$(m₂)/approxvsmcmc_kappa.png",
    a=a,
    b=b,
    step=step;
    xLabel="kappa_u",
    yLabel="density",
)

In [ ]:
# Pour μ
k = 1;

approxDensity(x::Real) = pdf(Normal(η[k], sqrt(s²[k])), x);
a = 9.5;
b = 11.5;
step = .001;

plotApproxVSMCMC(
    approxDensity,
    θsampling.value[:, k+1, 1],
    # "../plots/mean_field/$(m₁)x$(m₂)/approxvsmcmc_mu.png",
    a=a,
    b=b,
    step=step,
    xLabel="mu",
    yLabel="density",
)